# Drifting initial electrons in TPC

Based on electrons drift process from photocathode in Xenodiffusionscope.

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 

from xenodiffusionscope import Source
from xenodiffusionscope import TPC
from xenodiffusionscope import ElectronDrift 
from xenodiffusionscope import MeshGrid
from xenodiffusionscope import TopArray
from xenodiffusionscope import LCEPattern

import plot_utils as pu
from tqdm import tqdm

import nestpy

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Figure size and DPI
plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['figure.dpi'] = 100

# Font family and size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Utopia']
plt.rcParams['font.size'] = 28

# Mathtext font set
plt.rcParams['mathtext.fontset'] = 'cm'


### Initialise TPC and Source objects

In [4]:
r_max, hex_size = 75, 1.56
length = 2600
liquid_gap = 5
gas_gap = 5
drift_field = 100

In [5]:
Xenoscope = TPC(r_max, length, liquid_gap, gas_gap, drift_field)
source = Source('../data/Ba133_expanded5k.csv') # Load the input data 

## Initial electron positions 

In [ ]:
# Choose a specific interaction energy and electric field ADD UNITS
interaction = nestpy.nr
density = 2.9 
field = 100.

# Number of electrons generated from interactions
n_electrons = source.n_initial_electrons(interaction, field)
# Initial positions of electrons
positions = source.pos_initial_electrons(n_electrons)

In [ ]:
print(positions)

## Drifting initial electrons 

In [ ]:
# Initialise a drift simulator
drift = ElectronDrift(positions, Xenoscope)

# Drift the electrons to the gate and get the final positions
x, y , z = drift.drift_electrons()

In [ ]:
#Apply e-lifetime and extraction efficiency, electrons are suppressed 
x_corr_elifetime,y_corr_elifetime,z_corr_elifetime = drift.apply_elifetime(x,y,z)
x_extracted,y_extracted, z_extracted = drift.extract_electrons(x_corr_elifetime,y_corr_elifetime,z_corr_elifetime)

## Focus electrons on gate mesh 

In [ ]:
from hexalattice.hexalattice import create_hex_grid
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

In [ ]:
# Create a mesh object 
mesh = Xenoscope.gate_mesh

In [ ]:
#Focus electrons on hex centers
e_pos = np.stack((x_extracted, y_extracted, z_extracted), axis = 1)

pos_focus = mesh.focus_on_grid(e_pos)

pos_counts = mesh.count_focused(pos_focus)

counts_pe_on_hex = drift.convert_electron_to_photons(pos_counts)

In [ ]:
# how many hexagon we need?
r_max = 75
a = 1.56
r_hex = a * np.sqrt(3)

n_hex_x = 2 * np.ceil(75/r_hex)
n_hex_y = 2 * np.ceil(75/(3/2*a))

In [ ]:
# fig, ax = plt.subplots(1,1, figsize = (15,15))

# hex_centers, _ = create_hex_grid(nx=n_hex_x, ny=n_hex_y, 
               #                  min_diam = r_hex,crop_circ = r_max,
               #                  h_ax =ax, do_plot = True)

#hex_origin_dist = np.sqrt(np.power(hex_centers[:,0],2)+np.power(hex_centers[:,1],2))
#mask_r = hex_origin_dist < r_max
#hex_centers_inside = hex_centers[mask_r]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 9))

hex_centers, _ = create_hex_grid(nx=n_hex_x, ny=n_hex_y, min_diam=r_hex, crop_circ=r_max, h_ax=ax, do_plot=True)

ax.scatter(e_pos[:1000, 0], e_pos[:1000, 1], label='Unfocused', alpha=0.6, s=25, c = '#4C82C0')
ax.scatter(pos_focus[:1000, 0], pos_focus[:1000, 1], label='Focused', marker='x', linewidth=1.4, alpha=0.9, s=25, c = '#D92F41')

# TPC radius 
c = Circle((0,0), r_max, fill = False, color = '#FF4C48')
ax.add_patch(c)

# Set axis labels
ax.set_xlabel('x [mm]')
ax.set_ylabel('y [mm]')

# Set legend
ax.legend(loc='upper right')

# Set title
ax.set_title('Co57')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.5)

# Set background color
ax.set_facecolor('#FAFAFA')
        

# Save
save_path = "/home/atp/souaha/souahada/figures/XenoDiffusion/focusing_Co57.png"
plt.savefig(save_path, facecolor="white", bbox_inches='tight', pad_inches=0.1)

# Show the plot
plt.show()


In [ ]:
## To zoom in

fig, ax = plt.subplots(1,1, figsize = (10,10), dpi = 60)

hex_centers, h_ax = create_hex_grid(nx=n_hex_x, ny=n_hex_y, 
                                    min_diam = r_hex,crop_circ = r_max,
                                    h_ax =ax, do_plot = True)

ax.scatter(e_pos[:1000, 0], e_pos[:1000, 1], marker = '.',c = 'b', label = 'Initial positions', alpha = 0.5)
ax.scatter(pos_focus[:1000, 0], pos_focus[:1000, 1], c = 'r',marker = 'x', label = 'Final positions')

ax.set_xlim(-70,70)
ax.set_ylim(-70,70)
ax.set_xlabel('x [mm]')
ax.set_xlabel('y [mm]')
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

In [ ]:
# Plot number of electrons on each hex center 
pu.electrons_hist(mesh.n_hexes, pos_counts, 'Co57', 'hist_Co57')

## Pattern reconstruction

In [ ]:
pattern_path = '/disk/gfs_atp/xenoscope/toy-mc/LCE_patterns'

# Xenoscope, source and mesh already defined above. Later have in a separate notebook
top_array = TopArray(tpc = Xenoscope, 
                     mesh = mesh,
                     model = 'sixbysix',
                     path_to_patterns = pattern_path,
                    path_to_model = '../TopArrayModel/' )

In [ ]:
say_a_number = 50
pattern = top_array.load_pattern(say_a_number)

In [ ]:
LCEPattern.plot_pattern(Xenoscope,pattern,say_a_number) ## In TPC r max has been changed to radius

In [ ]:
top_array.load_all_patterns(all_at_once=True)

In [ ]:
top_array.fill_grid_from_events(counts_pe_on_hex)

In [ ]:
def plot_results(fig, ax,top_array):
    xx = top_array.grid_xx
    yy = top_array.grid_yy
    zz = top_array.grid_zz
    rr = TPC.get_r(xx,yy)
    xx = xx[rr < Xenoscope.radius]
    yy = yy[rr < Xenoscope.radius]
    zz = zz[rr < Xenoscope.radius]
    
    
    ax.set_title('Summed results')
    sc = ax.scatter(xx, yy,
                    c=np.log10(zz),
               marker = 's', s = 10, vmin = 0,alpha = 0.8)

    ax.add_patch(Circle((0,0),75, color = 'r',fill = False, linewidth = 1, ls ='--'))
    ax.set_aspect('equal')
    ax.set_xlabel('x [mm]')
    ax.set_ylabel('y [mm]')
    fig.colorbar(sc, ax = ax, label = 'pe$\cdot$mm$^{-2}$')
    
    ax = top_array.plot_toparray(ax)
    return fig,ax

In [ ]:
top_array.model

In [ ]:
import pickle
from matplotlib.patches import Circle


In [ ]:
#top_array.load_top_array()
with open('../TopArrayModel/%s.pck'%top_array.model, 'rb') as file:
    top_array.sensors = pickle.load(file)
top_array.n_sensors = len(top_array.sensors)

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (9,9), dpi = 100)
fig, ax = plot_results(fig, ax, top_array)
ax = top_array.plot_toparray(ax)

#plt.savefig('figures/results_with_array.png')
plt.show()

In [ ]:
import matplotlib as mpl

In [ ]:
top_array.load_all_patterns(all_at_once=True)
top_array.fill_grid_from_events(counts_pe_on_hex)
top_array.load_top_array()
n_pe_quads = top_array.n_pe_in_sensors()

In [ ]:
n_pe_quads

In [ ]:
Xenoscope.diffusion_trans

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

sunset = ['#364B9A', '#4A7BB7', '#6EA6CD', '#98CAE1', '#C2E4EF',
         '#EAECCC', '#FEDA8B', '#FDB366', '#F67E4B', '#DD3D2D',
         '#A50026']


# Define positions for custom heatmap
positions = np.linspace(0, 1, len(sunset)) 

# Create the custom colormap using LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list("Custom Colormap", list(zip(positions, sunset)))

In [ ]:

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

fig, ax = plt.subplots(1, 1, figsize=(5, 4), dpi=200)

# Assuming top_array is an instance of a custom class for plotting top arrays
ax = top_array.plot_toparray(ax, pe_in_sensors=True)

cmap = mpl.cm.inferno
norm = mpl.colors.Normalize(vmin=0, vmax=np.log10(np.max(n_pe_quads)))

# Set the array of values for the ScalarMappable
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([n_pe_quads])  # Replace the empty list with your actual data array

# Create the colorbar
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', label='log10(PE)', ticks=[1, 2, 3, 4])

ax.set_yticks((-50, 0, 50))
plt.title('6x6 layout')

plt.tight_layout()

plt.show()


In [ ]:
top_array = TopArray(tpc = Xenoscope, 
                     mesh = mesh,
                     model = 'quads',
                     path_to_patterns = pattern_path,
                    path_to_model = '../TopArrayModel/' )

In [ ]:

top_array.fill_grid_from_events(counts_pe_on_hex)
top_array.load_top_array()
n_pe_quads = top_array.n_pe_in_sensors()

In [ ]:

fig,ax = plt.subplots(1,1,figsize = (11,9), dpi = 100)
#fig, ax = plot_results(fig, ax, top_array, res)
ax = top_array.plot_toparray(ax,pe_in_sensors=True)

cmap = mpl.cm.inferno
norm = mpl.colors.Normalize(vmin=np.log10(np.min(n_pe_quads)), 
                            vmax=np.log10(np.max(n_pe_quads)))

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             ax=ax, orientation='vertical', label='log10(Photoelectrons detected)')

plt.show()
